In [2]:
import torch
import pickle
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModel

def create_claim_encodings(config_name,file_path,output_path):
    tokenizer = AutoTokenizer.from_pretrained(config_name)
    model = AutoModel.from_pretrained(config_name)
    with open(file_path,'r') as f:
        claim_descriptions = f.readlines()
    claim_descriptions[0] = claim_descriptions[0][1:]
    descriptions = clean_desc(claim_descriptions)
    tokens = tokenizer(descriptions, padding='max_length', max_length = 25, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)

    with open(output_path,'wb') as f:
        pickle.dump(outputs.last_hidden_state,f)
    
    return outputs.last_hidden_state

def clean_desc(claim_descriptions):
    desc_fin = []
    for i in range(len(claim_descriptions)):
        desc = claim_descriptions[i].strip().strip(',').strip('\"').lower().split()
        fin = []
        for wrd in desc:
            if wrd not in stopwords.words('english'):
                fin.append(wrd)
                
        fin = ' '.join(fin)
        desc_fin.append(fin)
    return desc_fin

In [3]:
encodes = create_claim_encodings('google/electra-base-discriminator','claim-descriptions-daberta_original.txt','electra_claim_desc_orig_vecs_base.pkl')

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
